In [1]:
# import Pkg
# Pkg.add("Peaks")

In [18]:
#=
Step 0: Open recording or generate signal
=#

# To Generate Signal:
include("../signal_generator/tone.jl")
include("../sensor.jl")

freq = 50;          # Frequency of Tone (in Hz)
amp = 1;            # Amplitude of Tone
duration = 10;       # Duration of Tone (in seconds)
sample_rate = 32000.0;
tone_sig, n = tone(duration, amp, freq, sample_rate);

include("../signal_generator/generate_sig.jl")
using DSP.Windows: hanning, rect, bartlett
az_gt = -90;        # Ground Truth Azimuth Angle (in degrees)
az_gt2 = 120;
c0 = 1500;          # Speed of Medium (in m/s)
NFFT = 2^11;
noverlap = Int(NFFT * (3 // 4));
sig1, sample_rate = simulate_sensor_signal(tone_sig, sample_rate, sensors_underwater,
                         NFFT, noverlap, hanning, az_gt, c0);
sig2, sample_rate = simulate_sensor_signal(tone_sig, sample_rate, sensors_underwater, 
                        NFFT, noverlap, hanning, az_gt2, c0); 
new_sig = sig1;

sensors Loaded: (8,)
sensors1 Loaded: (25,)
sensors2 Loaded: (20,)
sensors_underwater loaded: (40,)


Start Simulating Signal
Signal has size: (320000, 1)


  2.550662 seconds (1.81 M allocations: 1.528 GiB, 6.92% gc time, 42.54% compilation time)
Now Generated Signal has size: (320000, 40)
Start Simulating Signal
Signal has size: (320000, 1)


  1.632204 seconds (180.56 k allocations: 1.453 GiB, 19.98% gc time)
Now Generated Signal has size: (320000, 40)


In [19]:
include("../utils/preprocess.jl")
function filter_freq_per_ch(new_sig)
    new_S = []
    for sig in eachcol(new_sig)
        S_interest = choose_freq(sig, freq, sample_rate);
        push!(new_S, S_interest);
    end
    # test_sig = Matrix{}(undef, size(new_sig, 2)) 
    new_S = mapreduce(permutedims, vcat, new_S);
    return new_S
end

new_S = filter_freq_per_ch(new_sig);

In [20]:
using Statistics
n_snapshots = 128;
Sy = cov(new_S[:,1:n_snapshots], dims=2)

40×40 Matrix{ComplexF64}:
  9.1403e5+0.0im           9.1403e5-4.74748e-13im  …  8.98298e5-1.80951e5im
  9.1403e5+4.74748e-13im   9.1403e5+0.0im             8.98298e5-1.80951e5im
  9.1403e5+4.74748e-13im   9.1403e5+4.74748e-13im     8.98298e5-1.80951e5im
  9.1403e5+4.74748e-13im   9.1403e5+4.74748e-13im     8.98298e5-1.80951e5im
  9.1403e5+4.74748e-13im   9.1403e5+4.74748e-13im     8.98298e5-1.80951e5im
 9.10802e5+91194.0im      9.10802e5+91194.0im      …  9.13255e5-90688.0im
  9.0783e5-90896.3im       9.0783e5-90896.3im         8.74135e5-2.69056e5im
 8.92468e5-1.79776e5im    8.92468e5-1.79776e5im       8.41368e5-3.53365e5im
 9.07875e5-90517.5im      9.07875e5-90517.5im         8.74255e5-2.68692e5im
 8.92468e5-1.79776e5im    8.92468e5-1.79776e5im       8.41368e5-3.53365e5im
          ⋮                                        ⋱  
 8.70067e5+2.8995e5im     8.70067e5+2.8995e5im        9.12733e5+1.12712e5im
 9.10836e5+90812.8im      9.10836e5+90812.8im         9.13212e5-91069.2im
 8.98298e5+

In [21]:
include("../sensor.jl")
include("./cbf.jl")
using LinearAlgebra

sensors Loaded: (8,)
sensors1 Loaded: (25,)
sensors2 Loaded: (20,)
sensors_underwater loaded: (40,)


In [22]:
az_list = -180:180;
num_sensors = length(sensors_underwater);
A = Matrix{ComplexF64}(undef, num_sensors, length(az_list))
for (sensor_idx, sensor) in enumerate(sensors_underwater)
    for (az_idx, az) in enumerate(az_list)
        A[sensor_idx, az_idx] = vandermonde_weight(sensor, az, 90, freq, c0);
    end
end


In [23]:
# function SBL(Sy, A, K=1, ϵ = 0.001, σ_noise = 0.1, max_iter = 500)

ErrorException: syntax: incomplete: premature end of input

In [24]:
# SBL Algorithm
ϵ = 0.001; # Numerical Tolerance of Algorithm
γ_collection = Vector{Float64}(undef, length(az_list));
max_iter = 500;
K = 1;      # Number of Peaks
σ_noise = 0.1;
num_sensors = size(A, 1);

# Initialization
γ_collection, ___ = cbf(Sy, sensors_underwater, freq, c0);

for iter = 1:max_iter
    Γ = diagm(γ_collection);
    Σy = σ_noise .* I + A * Γ * A'; # Note: I is the identity Matrix

    # Start Finding new Amplitudes
    γ_new = Vector{ComplexF64}(undef, length(γ_collection));
    for (az_idx, az) in enumerate(az_list)
        a = A[:, az_idx];
        inv_y = inv(Σy);
        numo = tr(Sy * inv_y * a * a' * inv_y); # Is this PSD too by right?
        denom = a' * inv_y * a;         # Is this PSD by right?
        # println("Numerator: $(numo), Denomenator: $(denom)")
        γ_new[az_idx] = γ_collection[az_idx] .* abs(numo) / abs(denom);
    end
    γ_new = abs.(γ_new);

    # Find Peaks & Re-estimate Noise Parameter
    ___, az_max_idx = findmax(γ_new); # Technically Find local Peaks
    Am = A[:, az_max_idx];
    σ_noise = tr(I - Am * pinv(Am)) ./ (num_sensors - K);
    σ_noise = abs(σ_noise);

    err = norm(γ_new - γ_collection, 1) ./ norm(γ_collection, 1);
    println("At iter: $iter , Noise Value = $(σ_noise) w/ error: $err")
    # Stopping Criterion
    err < ϵ ? break : γ_collection = γ_new;
end

At iter: 1 , Noise Value = 1.0000000000000007 w/ error: 0.9290384467060918


At iter: 2 , Noise Value = 1.0000000000000007 w/ error: 0.9450877038438034


At iter: 3 , Noise Value = 1.0000000000000007 w/ error: 0.6185303896852284


At iter: 4 , Noise Value = 1.0000000000000007 w/ error: 0.5465666655896638
At iter: 5 , Noise Value = 1.0000000000000007 w/ error: 0.6050923618644355


At iter: 6 , Noise Value = 1.0000000000000007 w/ error: 0.7462658488887701


...

At iter: 34 , Noise Value = 1.0000000000000007 w/ error: 0.15998371761929545
At iter: 35 , Noise Value = 1.0000000000000007 w/ error: 0.08975900470360505


At iter: 36 , Noise Value = 1.0000000000000007 w/ error: 0.049229254373820565


At iter: 37 , Noise Value = 1.0000000000000007 w/ error: 0.028500630380664346


At iter: 38 , Noise Value = 1.0000000000000007 w/ error: 0.021505024737611542


At iter: 39 , Noise Value = 1.0000000000000007 w/ error: 0.03227875062671477


In [ ]:
plot(az_list, γ_collection)

In [ ]:
using Plots
P_db, az_max = predict_az(γ_collection, az_list);
plot(az_list, P_db)

In [ ]:
include("./dbf_freeform.jl")
order = 3;
ψ_mat = create_ψ_matrix(sensors_underwater, order, freq, c0);
v = ones(Rational, 2*order+1); # Weights associated with DBF
v .//= size(v,1);
sensor_term = ψ_mat' * inv( ψ_mat * ψ_mat' );
A = Matrix{ComplexF64}(undef, num_sensors, size(az_list,1))
for (idx, az) in enumerate(az_list)
    γ = steering_matrix(az, order);
    h_dbf = sensor_term * conj(γ) * v;
    A[:, idx] = h_dbf
end

In [ ]:
# SBL Algorithm
ϵ = 0.001; # Numerical Tolerance of Algorithm
γ_collection = Vector{Float64}(undef, length(az_list));
max_iter = 500;
K = 1;      # Number of Peaks
σ_noise = 0.1;
num_sensors = length(sensors_underwater);

# Initialization
γ_collection, ___ = dbf_coFree(Sy, sensors_underwater, order, freq, c0);

for iter = 1:max_iter
    Γ = diagm(γ_collection);
    Σy = σ_noise .* I + A * Γ * A'; # Note: I is the identity Matrix

    # Start Finding new Amplitudes
    γ_new = Vector{ComplexF64}(undef, length(γ_collection));
    for (az_idx, az) in enumerate(az_list)
        a = A[:, az_idx];
        inv_y = inv(Σy);
        numo = tr(Sy * inv_y * a * a' * inv_y); # Is this PSD too by right?
        denom = a' * inv_y * a;         # Is this PSD by right?
        # println("Numerator: $(numo), Denomenator: $(denom)")
        γ_new[az_idx] = γ_collection[az_idx] .* abs(numo) / abs(denom);
    end
    γ_new = abs.(γ_new);

    # Find Peaks & Re-estimate Noise Parameter
    ___, az_max_idx = findmax(γ_new); # Technically Find local Peaks
    Am = A[:, az_max_idx];
    σ_noise = tr(I - Am * pinv(Am)) ./ (num_sensors - K);
    σ_noise = abs(σ_noise);

    err = norm(γ_new - γ_collection, 1) ./ norm(γ_collection, 1);
    println("At iter: $iter , Noise Value = $(σ_noise) w/ error: $err")
    # Stopping Criterion
    err < ϵ ? break : γ_collection = γ_new;
end

In [ ]:
plot(az_list, γ_collection)

In [ ]:
using Plots
P_db, az_max = predict_az(γ_collection, az_list);
plot(az_list, P_db)

In [ ]:
using Peaks
findmaxima(γ_new)